# **Voice clone : Text to Speech Synthesis**

#### This project is about real human voice cloning. The main objective of our project to generate deepfake voice similar to original Human voice.Our project can be divided into two points: input audio from users and  clone voice. We have further classified them into seven steps which are given below.

     

*   Step 1:Import necessary libraries

*    Step 2:Usingt HTML and JS for microphone acces
   
*   Step 3:Record audio from users

*   Step 4:Saving audio files

*   Step 5:Clone the github project and download pretrained models

*   Step 6:Initialize the voice cloning models

*   Step 7:Synthesis text to audio for Voice Clone










    

**Record audio from Google colab using Javascript and HTML**

 First we install and import necessary libraries for recording audio .wav files from Users and access microphone from google colab

**Step 1:Import necessary libraries**

In [1]:
!pip install ffmpeg-python
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

**Step 2:Usingt HTML and JS for microphone acces**

In [2]:
AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };            
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {            
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data); 
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});
      
</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  
  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)
  
  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr

**Step 3:Record audio from users**

In [3]:
audio, sr = get_audio()

**Step 4:Saving audio files**

In [6]:
pip install wavio


In [7]:
import wavio


wavio.write("realvoice.wav", audio, sr)

**Real-Time-Voice-Cloning** 

In this step we clone files from [CorentinJ/Real-Time-Voice-Cloning](https://https://github.com/CorentinJ/Real-Time-Voice-Cloning.git),download pretrained models and finally text to speech synthesis to get deep fake voice of input audio.

**Step 5:Clone the github project and download pretrained models**

In [8]:
%tensorflow_version 1.x
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CorentinJ/Real-Time-Voice-Cloning.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # clone and install
  !git clone -q --recursive {git_repo_url}
  # install dependencies
  !cd {project_name} && pip install -q -r requirements.txt
  !pip install -q gdown
  !apt-get install -qq libportaudio2
  !pip install -q https://github.com/tugstugi/dl-colab-notebooks/archive/colab_utils.zip

  # download pretrained model
  !cd {project_name} && wget https://github.com/blue-fish/Real-Time-Voice-Cloning/releases/download/v1.0/pretrained.zip && unzip -o pretrained.zip



TensorFlow 1.x selected.
     |████████████████████████████████| 80 kB 4.0 MB/s 
     |████████████████████████████████| 676 kB 10.7 MB/s 
     |████████████████████████████████| 10.3 MB 35.4 MB/s 
     |████████████████████████████████| 14.5 MB 80 kB/s 
     |████████████████████████████████| 235 kB 70.7 MB/s 
     |████████████████████████████████| 8.3 MB 45.1 MB/s 
     |████████████████████████████████| 66 kB 4.0 MB/s 
     |████████████████████████████████| 1.1 MB 69.1 MB/s 
     |████████████████████████████████| 52 kB 644 kB/s 
     |████████████████████████████████| 59.9 MB 35 kB/s 
     |████████████████████████████████| 317 kB 67.7 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 1.15.2 requires gast==0.2.2, but you have gast 0.4.0 which is incompatible.
lucid 0.3.10 requires numpy<=1.19, but you have numpy 1.19.4 which is incompatible

**Step 6:Initialize the voice cloning models**

In [9]:
import sys
sys.path.append(project_name)

from IPython.display import display, Audio, clear_output
from IPython.utils import io
import ipywidgets as widgets
import numpy as np
import librosa
from dl_colab_notebooks.audio import record_audio, upload_audio

from synthesizer.inference import Synthesizer
from encoder import inference as encoder
from vocoder import inference as vocoder
from pathlib import Path

encoder.load_model(project_name / Path("encoder/saved_models/pretrained.pt"))
synthesizer = Synthesizer(project_name / Path("synthesizer/saved_models/pretrained/pretrained.pt"))
vocoder.load_model(project_name / Path("vocoder/saved_models/pretrained/pretrained.pt"))

Loaded encoder "pretrained.pt" trained to step 1564501
Synthesizer using device: cpu
Building Wave-RNN
Trainable Parameters: 4.481M
Loading model weights at Real-Time-Voice-Cloning/vocoder/saved_models/pretrained/pretrained.pt


**Step 7:Synthesis text to audio for Voice Clone**

In [10]:
text = "Hello How are you?"
in_fpath = Path("/content/realvoice.wav")
reprocessed_wav = encoder.preprocess_wav(in_fpath)
original_wav, sampling_rate = librosa.load(in_fpath)
preprocessed_wav = encoder.preprocess_wav(original_wav, sampling_rate)
embed = encoder.embed_utterance(preprocessed_wav)
with io.capture_output() as captured:
  specs = synthesizer.synthesize_spectrograms([text], [embed])
generated_wav = vocoder.infer_waveform(specs[0])
generated_wav = np.pad(generated_wav, (0, synthesizer.sample_rate), mode="constant")
display(Audio(generated_wav, rate=synthesizer.sample_rate))

{| ████████████████ 28500/28800 | Batch Size: 3 | Gen Rate: 0.9kHz | }

**Reference**


[08 Recording Audio from a Microphone - Introduction to Google Colaboratory (Colab) for Research](https://https://www.youtube.com/watch?v=4DGkgUffWxs)

[Recording Audio in the Browser Using the MediaStream Recorder API](https:/blog.addpipe.com/recording-audio-in-the-browser-using-pure-html5-and-minimal-javascript/)

[Convert blob to base64](https://stackoverflow.com/questions/18650168/convert-blob-to-base64/18650249#18650249)

[Easy audio capture with the MediaRecorder API](https://hacks.mozilla.org/2014/06/easy-audio-capture-with-the-mediarecorder-api/)

[Record to an Audio File using HTML5 and JS](https://air.ghost.io/recording-to-an-audio-file-using-html5-and-js/)

[Clone Any Voice Using Machine Learning | KNOWLEDGE DOCTOR | Mishu Dhar](https://https://www.youtube.com/watch?v=2NdhKkxjf2M)

[CorentinJ/Real-Time-Voice-Cloning](https://github.com/CorentinJ/Real-Time-Voice-Cloning)

[Now Clone anyone's voice - simple steps - Deepfake Voice Cloning Tutorial](https://https://www.youtube.com/watch?v=I-tpsOSy5l4)




